In [41]:
from re import compile
import numpy
from eis1600.processing.preprocessing import get_yml_and_miu_df

In [22]:
LABEL_DICT = {'B-Q': 0, 'I-Q': 1, 'O': 2}
CATS = ['N', 'T', 'O']
Q_PATTERN = compile(r"Q(?P<num_tokens>\d+)(?P<cat>[" + ''.join(CATS) + "]*)")

In [23]:
def reconstruct_automated_tag(row) -> str:
    return 'Q' + row['num_tokens']

In [28]:
file = '../../online-editor/data/production/tasks_output/Topo_Data/1389AghaBuzurgTihrani.TabaqatAclamShica.Masaha0003164Vols-ara1.939423474177.EIS1600'
with open(file, 'r', encoding='utf-8') as miu_file_object:
        yml_handler, df = get_yml_and_miu_df(miu_file_object)

s_notna = df['TAGS_LISTS'].loc[df['TAGS_LISTS'].notna()].apply(lambda tag_list: ','.join(tag_list))
df_true = s_notna.str.extract(Q_PATTERN).dropna(how='all')
tops = df_true.apply(reconstruct_automated_tag, axis=1)
tops.name = 'TRUE'

print(tops)

if not tops.empty:
    df = df.join(tops)
else:
    df['TRUE'] = nan

df

83    Q5
Name: TRUE, dtype: object


,SECTIONS,TOKENS,TAGS_LISTS,TRUE
0,_ء_#=939423474177= $,None,None,NaN
1,_ء_=823341916676= ::UNDEFINED:: ~,علي,None,NaN
2,None,بن,None,NaN
3,None,زيد,None,NaN
4,None,بن,None,NaN
...,...,...,...,...
435,None,من,None,NaN
436,None,العامة,None,NaN
437,None,،,None,NaN
438,None,وليس,None,NaN


In [36]:
if any(df['TRUE'].notna()):
    s_notna = df['TRUE'].loc[df['TRUE'].notna()]
    df_matches = s_notna.str.extract(Q_PATTERN).dropna(how='all')

    for index, row in df_matches.iterrows():
        processed_tokens = 0
        num_tokens = int(row['num_tokens'])
        cat = row['cat']
        while processed_tokens < num_tokens:
            if processed_tokens == 0:
                df.loc[index, 'BIO'] = 'B-' + 'Q' + cat
            else:
                df.loc[index + processed_tokens, 'BIO'] = 'I-' + 'Q' + cat

            processed_tokens += 1

    df["BIO"].loc[df["BIO"].isna()] = "O"

df

,SECTIONS,TOKENS,TAGS_LISTS,TRUE,BIO
0,_ء_#=939423474177= $,None,None,NaN,nan
1,_ء_=823341916676= ::UNDEFINED:: ~,علي,None,NaN,nan
2,None,بن,None,NaN,nan
3,None,زيد,None,NaN,nan
4,None,بن,None,NaN,nan
...,...,...,...,...,...
435,None,من,None,NaN,nan
436,None,العامة,None,NaN,nan
437,None,،,None,NaN,nan
438,None,وليس,None,NaN,nan


In [46]:
df['BIO'].dtypes

dtype('O')

In [43]:
df["BIO_IDS"] = df["BIO"].apply(lambda bio_tag: LABEL_DICT[bio_tag])
idcs = df["TOKENS"].loc[df["TOKENS"].notna()].index

KeyError: 'nan'